### Test ConsLinUCB, LinUCB, ThresholdBandit and GreedyBandit

In [1]:
%cd /home/lansdell/projects/conservativerdd
%pylab inline
pylab.rcParams['figure.figsize'] = (6, 6)

from lib.bandits import LinUCB, ThresholdBandit,ThresholdConsBandit,GreedyBandit, ConsLinUCB, expected_regret, expected_regret_per_arm
from lib.generator import LinearGeneratorParams, LinearGenerator
import numpy as np 

import matplotlib.pyplot as plt
from scipy.stats import truncnorm
import seaborn as sns
import pandas as pd
sns.set_style('ticks')

/home/lansdell/projects/conservativerdd
Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python2.7/dist-packages/pandas/core/computation/__init__.py:18: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)
/home/lansdell/.local/lib/python2.7/site-packages/matplotlib/__init__.py:913: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
#alg = 'greedy'
#alg = 'linucb'
#alg = 'threshold'
#alg = 'thresholdcons'
alg = 'conslinucb'

M = 3    #number of runs
N = 10000 #number of timesteps
save = False
alpha = 0.1 #CLUCB parameter...
#alpha = 1.0 #CLUCB parameter...
max_alpha = 2
max_beta = 2
k = 4    #Number of arms
d = 5   #Dimension of context (includes one dim for intercept term)
intercept = True
evaluate_every = 100

if alg == 'greedy':
	BanditAlg = GreedyBandit
elif alg == 'linucb':
	BanditAlg = LinUCB
elif alg == 'threshold':
	BanditAlg = ThresholdBandit
elif alg == 'thresholdcons':
	BanditAlg = ThresholdConsBandit
elif alg == 'conslinucb':
	BanditAlg = ConsLinUCB
else:
	print("Select a valid algorithm")
    
baseline_idx = 0

In [3]:
def second_largest_idx(a):
    vals = a.sort()

## Run CLUCB 

In [4]:
#Generate slopes and intercepts
alphas = truncnorm.rvs(-max_alpha, max_alpha, scale = 1, size=(M,k,d-1))
betas = truncnorm.rvs(-max_beta, max_beta, scale = 1, size=(M,k))

#baseline_alphas = truncnorm.rvs(-max_alpha, max_alpha, scale = 1, size=(M,1,d-1))
#baseline_betas = truncnorm.rvs(-max_beta, max_beta, scale = 1, size=(M,1))

regret = np.zeros((M, N))
expt_regret = np.zeros((M, N))
arm_pulls = np.zeros((M, N, k+1))
n_changes = np.zeros((M, N))
update_pol = np.zeros((M, N))

In [5]:
print("Running %s algorithm"%alg)
expt_rewards_clucb = np.zeros((M, N))
expt_rewards_baseline_clucb = np.zeros((M, N))
BanditAlg = ConsLinUCB

for m in range(M):
    params = LinearGeneratorParams(np.atleast_2d(alphas[m,:,:]), betas[m,:], d = d, k = k, intercept = intercept)
    generator = LinearGenerator(params)
    means = expected_regret_per_arm(generator)
    #Choose the baseline arm as the 2nd best arm on average....
    print(means)
    sorted_means = np.sort(means)
    #m_idx = np.argmax(means)
    m_idx = np.where(means == sorted_means[baseline_idx])[0][0]
    print('Baseline arm is %d'%m_idx)
    base_alpha = alphas[m,m_idx,:]
    base_beta = betas[m,m_idx]
    bandit = BanditAlg(generator, (base_alpha, base_beta), alpha = alpha)
    print("Run: %d/%d"%(m+1,M))
    for i in range(N):
        (ctx, arm_idx, obs, r) = bandit.step()
        regret[m,i] = r
        print(i, arm_idx, r)
        if arm_idx >= 0:
            arm_pulls[m,i,arm_idx] = 1
            expt_rewards_clucb[m,i] = np.dot(ctx[1:], alphas[m,arm_idx,:]) + betas[m,arm_idx]
        else:
            arm_pulls[m,i,m_idx] = 1
            expt_rewards_clucb[m,i] = np.dot(ctx[1:], alphas[m,m_idx,:]) + betas[m,m_idx]
        if arm_idx == m_idx:
            print("idx: %d -- playing baseline arm"%i)
        expt_rewards_baseline_clucb[m,i] = np.dot(ctx[1:],base_alpha) + base_beta


Running conslinucb algorithm
[ 0.33695143  0.91423609  0.24026951  1.84036119]
Baseline arm is 2
Run: 1/3
(0, -1, 0.19105397234112587)
(1, 0, 0.6418908149083864)
(2, -1, 0.48128937082160905)
(3, -1, 0.29376418529604209)
(4, -1, 0.0)
(5, -1, 0.0)
(6, -1, 0.0)
(7, -1, 0.0)
(8, -1, 0.4593446022115355)
(9, -1, 1.1505009914067603)
(10, -1, 0.029393871355637369)
(11, -1, 0.50106795261829729)
(12, -1, 0.051145976864328979)
(13, -1, 0.0)
(14, -1, 0.0)
(15, -1, 0.61565793739234154)
(16, -1, 0.066038959459166657)
(17, -1, 0.2999072585576919)
(18, -1, 0.0)
(19, -1, 0.0)
(20, -1, 0.0)
(21, -1, 0.25923504853411233)
(22, -1, 0.3505233399302643)
(23, -1, 0.0)
(24, -1, 0.98501789275517548)
(25, -1, 0.59365276767194775)
(26, -1, 0.33134790892750454)
(27, -1, 0.0)
(28, -1, 0.35279519186225761)
(29, -1, 0.044686308517508766)
(30, -1, 0.20160338364830693)
(31, -1, 0.0)
(32, -1, 0.0)
(33, -1, 0.21554254863357453)
(34, -1, 0.0)
(35, -1, 1.0536554322250764)
(36, -1, 0.78135922564571259)
(37, -1, 0.0)
(38, -1

(361, -1, 0.0)
(362, -1, 0.6372025286517663)
(363, -1, 1.3636972886887802)
(364, -1, 0.0)
(365, -1, 0.0)
(366, -1, 0.086847573802954936)
(367, -1, 0.17137575822050205)
(368, -1, 0.99709595410684171)
(369, -1, 0.4118606412562541)
(370, -1, 0.25583070063608521)
(371, -1, 0.84102319439294804)
(372, -1, 0.0)
(373, -1, 0.0)
(374, -1, 0.18606534992811347)
(375, -1, 0.0)
(376, -1, 0.5078482014449377)
(377, -1, 0.0)
(378, -1, 0.35704789392138697)
(379, -1, 0.0)
(380, -1, 0.41670419095773664)
(381, -1, 0.86903237112553566)
(382, -1, 1.1703204845781792)
(383, -1, 0.0)
(384, -1, 0.0)
(385, -1, 0.73531367190847363)
(386, -1, 0.0)
(387, -1, 0.26830203672860331)
(388, 3, 1.0191593062974695)
(389, -1, 0.47498501610910571)
(390, -1, 0.62733974602261344)
(391, -1, 0.0)
(392, -1, 1.0664671400040797)
(393, -1, 0.19352444424792831)
(394, -1, 0.0)
(395, -1, 0.69733865225167024)
(396, -1, 0.21905546966584821)
(397, -1, 0.38782723085366166)
(398, -1, 0.048051950055981674)
(399, -1, 0.33292512667668428)
(400,

(799, -1, 0.0)
(800, -1, 0.41095097780297152)
(801, -1, 0.15673050982458825)
(802, -1, 0.0)
(803, -1, 0.34443500662574184)
(804, -1, 0.45250386489622951)
(805, -1, 0.47893704472078735)
(806, -1, 0.71217825319360961)
(807, -1, 0.46900256734699963)
(808, -1, 0.0024527938013605977)
(809, -1, 0.87487414425133847)
(810, -1, 0.0)
(811, -1, 0.0)
(812, -1, 0.41506688918232204)
(813, -1, 0.0)
(814, -1, 0.0)
(815, -1, 0.31868727266949182)
(816, -1, 0.39124807328454625)
(817, -1, 0.0)
(818, -1, 0.0)
(819, -1, 0.027378843388640606)
(820, -1, 1.3275655012367165)
(821, -1, 0.0)
(822, -1, 0.68593535067525435)
(823, -1, 0.0)
(824, -1, 0.26645424725164396)
(825, -1, 0.0)
(826, -1, 0.071006043848293343)
(827, -1, 0.33882461554022081)
(828, -1, 0.59796652118820204)
(829, -1, 1.2479062283816806)
(830, -1, 0.0)
(831, -1, 0.0)
(832, -1, 0.0)
(833, -1, 0.42269839987235536)
(834, -1, 0.57909187860234268)
(835, -1, 0.81932393963241346)
(836, -1, 0.79659141126519517)
(837, -1, 0.29616418350995033)
(838, -1, 0.0

(1146, -1, 0.53368248783463268)
(1147, -1, 0.37581879336309137)
(1148, -1, 0.0)
(1149, -1, 0.13213208042540819)
(1150, -1, 0.93032672556203777)
(1151, -1, 0.0)
(1152, -1, 0.018537856138322262)
(1153, -1, 0.0)
(1154, -1, 0.0)
(1155, -1, 1.5161062988841425)
(1156, -1, 0.0)
(1157, -1, 0.96381605024140726)
(1158, -1, 0.31014160075239317)
(1159, -1, 0.0)
(1160, -1, 0.0)
(1161, -1, 0.012479925239698542)
(1162, -1, 0.34104435487832213)
(1163, -1, 0.0)
(1164, -1, 0.28150045435681376)
(1165, -1, 0.15178371667516183)
(1166, -1, 0.20507847752659875)
(1167, -1, 0.42669377507834833)
(1168, -1, 0.0)
(1169, -1, 0.0)
(1170, -1, 0.14149010816483593)
(1171, -1, 0.0)
(1172, -1, 0.41612414791978575)
(1173, -1, 0.0)
(1174, -1, 0.0)
(1175, -1, 0.031868457341337031)
(1176, -1, 0.0)
(1177, -1, 0.0)
(1178, -1, 0.0)
(1179, -1, 0.3522516412457164)
(1180, -1, 0.17543582219110504)
(1181, -1, 0.0)
(1182, -1, 1.1894265571880993)
(1183, -1, 0.42154313192226778)
(1184, -1, 0.41637498068981815)
(1185, -1, 0.0)
(1186, -

(1555, -1, 0.0)
(1556, -1, 0.0)
(1557, -1, 0.29112691275313196)
(1558, -1, 0.0)
(1559, -1, 0.0)
(1560, -1, 0.0)
(1561, -1, 0.0)
(1562, -1, 0.0)
(1563, -1, 0.27255932951421391)
(1564, -1, 0.0)
(1565, -1, 0.10133419923148446)
(1566, -1, 0.47112307290981742)
(1567, -1, 0.4982225814026115)
(1568, -1, 0.5316651508652176)
(1569, -1, 0.18652011581764072)
(1570, -1, 0.0)
(1571, -1, 0.0)
(1572, -1, 0.91700969364791318)
(1573, -1, 0.087715786105954485)
(1574, -1, 0.034388106783635186)
(1575, -1, 0.0)
(1576, -1, 0.0)
(1577, -1, 0.0)
(1578, -1, 0.0)
(1579, -1, 0.0)
(1580, -1, 0.56605894151153613)
(1581, -1, 0.44135963365698377)
(1582, -1, 0.0)
(1583, -1, 0.54099976613939971)
(1584, -1, 0.0096084024347639074)
(1585, -1, 0.0)
(1586, -1, 0.4884523487226351)
(1587, -1, 0.47458791996865823)
(1588, -1, 0.0)
(1589, -1, 0.87101344465837238)
(1590, -1, 0.49099237543018492)
(1591, -1, 0.22758279716543495)
(1592, -1, 0.011381879745529455)
(1593, -1, 0.15984360004788911)
(1594, -1, 0.0)
(1595, -1, 0.0)
(1596,

...

(8456, 2, 0.47603457810768024)
(8457, 2, 0.2818458382664214)
(8458, 0, 0.0)
idx: 8458 -- playing baseline arm
(8459, 3, 0.0)
(8460, 3, 0.0)
(8461, 3, 0.0)
(8462, 0, 0.0)
idx: 8462 -- playing baseline arm
(8463, 1, 0.0)
(8464, 1, 0.0)
(8465, 3, 0.0)
(8466, 0, 0.0)
idx: 8466 -- playing baseline arm
(8467, 3, 0.0)
(8468, 3, 0.0)
(8469, 0, 0.0)
idx: 8469 -- playing baseline arm
(8470, 3, 0.0)
(8471, 3, 0.0)
(8472, 0, 0.0)
idx: 8472 -- playing baseline arm
(8473, 0, 0.0)
idx: 8473 -- playing baseline arm
(8474, 1, 0.15900417017144619)
(8475, 1, 0.0)
(8476, 0, 0.0)
idx: 8476 -- playing baseline arm
(8477, 0, 0.0)
idx: 8477 -- playing baseline arm
(8478, 3, 0.0)
(8479, 1, 0.060161775352380042)
(8480, 3, 0.0)
(8481, 3, 0.0)
(8482, 0, 0.0)
idx: 8482 -- playing baseline arm
(8483, 3, 0.0)
(8484, 3, 0.0)
(8485, 0, 0.0)
idx: 8485 -- playing baseline arm
(8486, 3, 0.0)
(8487, 0, 0.0)
idx: 8487 -- playing baseline arm
(8488, 0, 0.0)
idx: 8488 -- playing baseline arm
(8489, 1, 0.0)
(8490, 1, 0.0)
(84

(8744, 3, 0.0)
(8745, 1, 0.0)
(8746, 3, 0.0)
(8747, 0, 0.0)
idx: 8747 -- playing baseline arm
(8748, 3, 0.0)
(8749, 0, 0.0)
idx: 8749 -- playing baseline arm
(8750, 3, 0.0)
(8751, 0, 0.0)
idx: 8751 -- playing baseline arm
(8752, 3, 0.0)
(8753, 0, 0.0)
idx: 8753 -- playing baseline arm
(8754, 3, 0.0)
(8755, 0, 0.0)
idx: 8755 -- playing baseline arm
(8756, 0, 0.0)
idx: 8756 -- playing baseline arm
(8757, 2, 0.72693437807177652)
(8758, 3, 0.0)
(8759, 0, 0.0)
idx: 8759 -- playing baseline arm
(8760, 3, 0.0)
(8761, 3, 0.0)
(8762, 3, 0.0)
(8763, 2, 0.5116653488410553)
(8764, 0, 0.0)
idx: 8764 -- playing baseline arm
(8765, 0, 0.0)
idx: 8765 -- playing baseline arm
(8766, 3, 0.0)
(8767, 0, 0.0)
idx: 8767 -- playing baseline arm
(8768, 1, 0.0)
(8769, 3, 0.0)
(8770, 1, 0.0)
(8771, 1, 0.0)
(8772, 0, 0.0)
idx: 8772 -- playing baseline arm
(8773, 3, 0.0)
(8774, 3, 0.0)
(8775, 1, 0.17975455227815945)
(8776, 0, 0.0)
idx: 8776 -- playing baseline arm
(8777, 1, 0.0)
(8778, 3, 0.0)
(8779, 1, 0.0)
(8780

(9062, 1, 0.0)
(9063, 0, 0.0)
idx: 9063 -- playing baseline arm
(9064, 3, 0.0)
(9065, 0, 0.0)
idx: 9065 -- playing baseline arm
(9066, 0, 0.0)
idx: 9066 -- playing baseline arm
(9067, 0, 0.0)
idx: 9067 -- playing baseline arm
(9068, 0, 0.0)
idx: 9068 -- playing baseline arm
(9069, 1, 0.0)
(9070, 1, 0.0)
(9071, 3, 0.0)
(9072, 0, 0.0)
idx: 9072 -- playing baseline arm
(9073, 0, 0.0)
idx: 9073 -- playing baseline arm
(9074, 0, 0.0)
idx: 9074 -- playing baseline arm
(9075, 3, 0.0)
(9076, 3, 0.0)
(9077, 1, 0.0)
(9078, 1, 0.15580439669490853)
(9079, 3, 0.012916371236142421)
(9080, 1, 0.0)
(9081, 1, 0.0)
(9082, 0, 0.0)
idx: 9082 -- playing baseline arm
(9083, 0, 0.0)
idx: 9083 -- playing baseline arm
(9084, 2, 0.42412321718983381)
(9085, 0, 0.0)
idx: 9085 -- playing baseline arm
(9086, 3, 0.0)
(9087, 3, 0.0)
(9088, 2, 0.31646992616443176)
(9089, 0, 0.0)
idx: 9089 -- playing baseline arm
(9090, 0, 0.0)
idx: 9090 -- playing baseline arm
(9091, 3, 0.0)
(9092, 3, 0.0)
(9093, 3, 0.0)
(9094, 1, 0.0

(9421, 0, 0.0)
idx: 9421 -- playing baseline arm
(9422, 0, 0.0)
idx: 9422 -- playing baseline arm
(9423, 1, 0.0)
(9424, 0, 0.0)
idx: 9424 -- playing baseline arm
(9425, 0, 0.0)
idx: 9425 -- playing baseline arm
(9426, 1, 0.0)
(9427, 0, 0.0)
idx: 9427 -- playing baseline arm
(9428, 0, 0.0)
idx: 9428 -- playing baseline arm
(9429, 3, 0.0)
(9430, 1, 0.0)
(9431, 0, 0.0)
idx: 9431 -- playing baseline arm
(9432, 1, 0.0)
(9433, 1, 0.0)
(9434, 3, 0.0)
(9435, 3, 0.0)
(9436, 3, 0.0)
(9437, 0, 0.0)
idx: 9437 -- playing baseline arm
(9438, 1, 0.0)
(9439, 0, 0.0)
idx: 9439 -- playing baseline arm
(9440, 3, 0.0)
(9441, 1, 0.073029541832926426)
(9442, 1, 0.1795940203147024)
(9443, 0, 0.0)
idx: 9443 -- playing baseline arm
(9444, 0, 0.0)
idx: 9444 -- playing baseline arm
(9445, 3, 0.0)
(9446, 3, 0.0)
(9447, 3, 0.048132755164966012)
(9448, 1, 0.0)
(9449, 0, 0.0)
idx: 9449 -- playing baseline arm
(9450, 0, 0.0)
idx: 9450 -- playing baseline arm
(9451, 0, 0.0)
idx: 9451 -- playing baseline arm
(9452, 3, 

(9720, 3, 0.0)
(9721, 3, 0.0)
(9722, 0, 0.0)
idx: 9722 -- playing baseline arm
(9723, 0, 0.0)
idx: 9723 -- playing baseline arm
(9724, 3, 0.0)
(9725, 0, 0.0)
idx: 9725 -- playing baseline arm
(9726, 0, 0.0)
idx: 9726 -- playing baseline arm
(9727, 3, 0.0)
(9728, 0, 0.0)
idx: 9728 -- playing baseline arm
(9729, 2, 0.40366022971404653)
(9730, 1, 0.23896946182935686)
(9731, 0, 0.0)
idx: 9731 -- playing baseline arm
(9732, 0, 0.0)
idx: 9732 -- playing baseline arm
(9733, 1, 0.0)
(9734, 0, 0.0)
idx: 9734 -- playing baseline arm
(9735, 3, 0.0)
(9736, 0, 0.0)
idx: 9736 -- playing baseline arm
(9737, 0, 0.0)
idx: 9737 -- playing baseline arm
(9738, 1, 0.0)
(9739, 3, 0.0)
(9740, 3, 0.0)
(9741, 0, 0.0)
idx: 9741 -- playing baseline arm
(9742, 0, 0.0)
idx: 9742 -- playing baseline arm
(9743, 0, 0.0)
idx: 9743 -- playing baseline arm
(9744, 0, 0.0)
idx: 9744 -- playing baseline arm
(9745, 1, 0.27849904329152042)
(9746, 1, 0.011737874564571704)
(9747, 0, 0.0)
idx: 9747 -- playing baseline arm
(9748

## LinUCB for comparison

In [6]:
BanditAlg = LinUCB    
regret_linucb = np.zeros((M, N))
arm_pulls_linucb = np.zeros((M, N, k+1))
expt_rewards_linucb = np.zeros((M, N))
expt_rewards_baseline_linucb = np.zeros((M, N))

In [7]:
for m in range(M):
    params = LinearGeneratorParams(np.atleast_2d(alphas[m,:,:]), betas[m,:], d = d, k = k, intercept = intercept)
    generator = LinearGenerator(params)
    means = expected_regret_per_arm(generator)
    #Choose the baseline arm as the worst arm....
    sorted_means = np.sort(means)
    #m_idx = np.argmax(means)
    m_idx = np.where(means == sorted_means[baseline_idx])[0][0]
    base_alpha = alphas[m,m_idx,:]
    base_beta = betas[m,m_idx]
    bandit = BanditAlg(generator)
    print("Run: %d/%d"%(m+1,M))
    print('Worst arm is %d'%m_idx)
    for i in range(N):
        (ctx, arm_idx, obs, r) = bandit.step()
        regret_linucb[m,i] = r
        print(i, arm_idx, r)
        if arm_idx >= 0:
            arm_pulls_linucb[m,i,arm_idx] = 1
        else:
            arm_pulls_linucb[m,i,k] = 1
        if arm_idx == m_idx:
            print("idx: %d -- playing baseline arm"%i)
        expt_rewards_baseline_linucb[m,i] = np.dot(ctx[1:],base_alpha) + base_beta
        expt_rewards_linucb[m,i] = np.dot(ctx[1:], alphas[m,arm_idx,:]) + betas[m,arm_idx]


Run: 1/3
Worst arm is 2
(0, 0, 0.30093243747283482)
(1, 1, 0.88653507570802492)
(2, 2, 0.2842436147421758)
idx: 2 -- playing baseline arm
(3, 3, 1.5955273516955919)
(4, 0, 0.0)
(5, 2, 0.0)
idx: 5 -- playing baseline arm
(6, 0, 0.05582130223273668)
(7, 3, 1.3618282444711722)
(8, 1, 0.0)
(9, 2, 0.0)
idx: 9 -- playing baseline arm
(10, 2, 0.0)
idx: 10 -- playing baseline arm
(11, 2, 0.54391915583120254)
idx: 11 -- playing baseline arm
(12, 1, 1.7916939385100168)
(13, 0, 0.3458847208110965)
(14, 1, 1.7464367852356046)
(15, 3, 2.0314466153912276)
(16, 2, 0.56749259641534033)
idx: 16 -- playing baseline arm
(17, 0, 0.91130180470138933)
(18, 2, 0.34314928645204906)
idx: 18 -- playing baseline arm
(19, 0, 0.0)
(20, 2, 0.0)
idx: 20 -- playing baseline arm
(21, 2, 0.5161784851565927)
idx: 21 -- playing baseline arm
(22, 1, 0.16480090106562584)
(23, 0, 0.0)
(24, 0, 0.21813666253789271)
(25, 0, 0.70247565681461532)
(26, 3, 0.55449856208792592)
(27, 1, 0.32517213863523892)
(28, 2, 0.247159899600041

idx: 348 -- playing baseline arm
(349, 0, 0.0)
(350, 0, 0.5411065484597648)
(351, 0, 0.0)
(352, 3, 1.513325103271717)
(353, 1, 0.0)
(354, 0, 0.0)
(355, 2, 0.29203761566361486)
idx: 355 -- playing baseline arm
(356, 2, 0.11516592411669568)
idx: 356 -- playing baseline arm
(357, 1, 0.1167193897000201)
(358, 0, 0.0)
(359, 0, 0.0)
(360, 2, 0.0)
idx: 360 -- playing baseline arm
(361, 0, 0.0)
(362, 0, 0.0)
(363, 0, 0.052038462435686217)
(364, 2, 0.30734182760546941)
idx: 364 -- playing baseline arm
(365, 1, 0.0)
(366, 0, 0.0)
(367, 1, 0.47416534452602888)
(368, 2, 0.0)
idx: 368 -- playing baseline arm
(369, 2, 0.067959487866343538)
idx: 369 -- playing baseline arm
(370, 0, 0.0)
(371, 2, 0.0)
idx: 371 -- playing baseline arm
(372, 2, 0.0)
idx: 372 -- playing baseline arm
(373, 0, 0.0)
(374, 0, 0.0)
(375, 3, 1.6313858647098494)
(376, 2, 0.0)
idx: 376 -- playing baseline arm
(377, 1, 1.5125354471720471)
(378, 1, 0.2619962907665509)
(379, 2, 0.0)
idx: 379 -- playing baseline arm
(380, 0, 0.0)
(3

(899, 2, 0.0)
idx: 899 -- playing baseline arm
(900, 2, 0.0)
idx: 900 -- playing baseline arm
(901, 2, 0.0082266675391717836)
idx: 901 -- playing baseline arm
(902, 2, 0.0)
idx: 902 -- playing baseline arm
(903, 2, 0.0)
idx: 903 -- playing baseline arm
(904, 1, 0.036489411763334756)
(905, 3, 0.43944931995147052)
(906, 1, 0.0)
(907, 1, 0.0)
(908, 2, 0.0)
idx: 908 -- playing baseline arm
(909, 0, 0.0)
(910, 2, 0.0)
idx: 910 -- playing baseline arm
(911, 1, 0.29156514049465448)
(912, 0, 0.0)
(913, 1, 0.0)
(914, 0, 0.0)
(915, 0, 0.0)
(916, 1, 0.20199061256655271)
(917, 2, 0.0)
idx: 917 -- playing baseline arm
(918, 0, 0.0)
(919, 2, 0.0)
idx: 919 -- playing baseline arm
(920, 0, 0.34355236095199454)
(921, 0, 0.0)
(922, 0, 0.0)
(923, 1, 0.0)
(924, 3, 1.2278784364216535)
(925, 0, 0.0)
(926, 1, 0.12497844637449024)
(927, 0, 0.0)
(928, 3, 0.16855461634062319)
(929, 1, 0.55969218681630051)
(930, 2, 0.0)
idx: 930 -- playing baseline arm
(931, 1, 0.10602158298271214)
(932, 0, 0.0)
(933, 2, 0.0)
id

(1393, 2, 0.0)
idx: 1393 -- playing baseline arm
(1394, 0, 0.0)
(1395, 2, 0.0)
idx: 1395 -- playing baseline arm
(1396, 0, 0.0)
(1397, 1, 0.0)
(1398, 2, 0.0)
idx: 1398 -- playing baseline arm
(1399, 2, 0.0)
idx: 1399 -- playing baseline arm
(1400, 2, 0.0)
idx: 1400 -- playing baseline arm
(1401, 0, 0.0)
(1402, 0, 0.0)
(1403, 0, 0.0)
(1404, 0, 0.0)
(1405, 0, 0.0)
(1406, 2, 0.0)
idx: 1406 -- playing baseline arm
(1407, 2, 0.0)
idx: 1407 -- playing baseline arm
(1408, 0, 0.0)
(1409, 1, 0.0)
(1410, 0, 0.0)
(1411, 1, 0.056495892762940314)
(1412, 0, 0.0)
(1413, 0, 0.0)
(1414, 0, 0.0)
(1415, 1, 0.039252492088099)
(1416, 2, 0.0)
idx: 1416 -- playing baseline arm
(1417, 3, 1.3338833889096116)
(1418, 0, 0.0)
(1419, 2, 0.0)
idx: 1419 -- playing baseline arm
(1420, 0, 0.0)
(1421, 0, 0.0)
(1422, 2, 0.0)
idx: 1422 -- playing baseline arm
(1423, 0, 0.0)
(1424, 0, 0.0)
(1425, 2, 0.0)
idx: 1425 -- playing baseline arm
(1426, 2, 0.0)
idx: 1426 -- playing baseline arm
(1427, 0, 0.0)
(1428, 0, 0.0)
(1429,

(1721, 1, 0.0)
(1722, 1, 0.0)
(1723, 3, 0.87257501851598052)
(1724, 2, 0.0)
idx: 1724 -- playing baseline arm
(1725, 2, 0.0)
idx: 1725 -- playing baseline arm
(1726, 2, 0.0)
idx: 1726 -- playing baseline arm
(1727, 0, 0.0)
(1728, 0, 0.0)
(1729, 2, 0.0)
idx: 1729 -- playing baseline arm
(1730, 2, 0.0)
idx: 1730 -- playing baseline arm
(1731, 0, 0.0)
(1732, 1, 0.0)
(1733, 2, 0.0)
idx: 1733 -- playing baseline arm
(1734, 0, 0.0)
(1735, 1, 0.087266433137872523)
(1736, 2, 0.0)
idx: 1736 -- playing baseline arm
(1737, 0, 0.0)
(1738, 1, 0.0)
(1739, 2, 0.0)
idx: 1739 -- playing baseline arm
(1740, 0, 0.20560042072464707)
(1741, 0, 0.15927062846772105)
(1742, 2, 0.0)
idx: 1742 -- playing baseline arm
(1743, 1, 0.025585272527354708)
(1744, 0, 0.0)
(1745, 0, 0.0)
(1746, 1, 0.0)
(1747, 0, 0.0)
(1748, 2, 0.0)
idx: 1748 -- playing baseline arm
(1749, 1, 1.0658025863874296)
(1750, 2, 0.0)
idx: 1750 -- playing baseline arm
(1751, 2, 0.0)
idx: 1751 -- playing baseline arm
(1752, 2, 0.0)
idx: 1752 -- pl

...

(8267, 0, 0.0)
idx: 8267 -- playing baseline arm
(8268, 0, 0.0)
idx: 8268 -- playing baseline arm
(8269, 0, 0.0)
idx: 8269 -- playing baseline arm
(8270, 0, 0.0)
idx: 8270 -- playing baseline arm
(8271, 3, 0.0)
(8272, 1, 0.070056956049857066)
(8273, 3, 0.0)
(8274, 0, 0.0)
idx: 8274 -- playing baseline arm
(8275, 3, 0.0)
(8276, 0, 0.0)
idx: 8276 -- playing baseline arm
(8277, 0, 0.0)
idx: 8277 -- playing baseline arm
(8278, 0, 0.0)
idx: 8278 -- playing baseline arm
(8279, 0, 0.0)
idx: 8279 -- playing baseline arm
(8280, 0, 0.0)
idx: 8280 -- playing baseline arm
(8281, 0, 0.0)
idx: 8281 -- playing baseline arm
(8282, 3, 0.0)
(8283, 0, 0.0)
idx: 8283 -- playing baseline arm
(8284, 1, 0.052653727540635797)
(8285, 0, 0.0)
idx: 8285 -- playing baseline arm
(8286, 3, 0.0)
(8287, 0, 0.0)
idx: 8287 -- playing baseline arm
(8288, 3, 0.0)
(8289, 0, 0.0)
idx: 8289 -- playing baseline arm
(8290, 3, 0.0)
(8291, 0, 0.0)
idx: 8291 -- playing baseline arm
(8292, 3, 0.0)
(8293, 3, 0.0)
(8294, 3, 0.0)
(8

(8560, 3, 0.0)
(8561, 0, 0.0)
idx: 8561 -- playing baseline arm
(8562, 3, 0.0)
(8563, 2, 0.82069956868859406)
(8564, 3, 0.0)
(8565, 0, 0.0)
idx: 8565 -- playing baseline arm
(8566, 3, 0.0)
(8567, 0, 0.0)
idx: 8567 -- playing baseline arm
(8568, 0, 0.0)
idx: 8568 -- playing baseline arm
(8569, 0, 0.0)
idx: 8569 -- playing baseline arm
(8570, 0, 0.0)
idx: 8570 -- playing baseline arm
(8571, 0, 0.0)
idx: 8571 -- playing baseline arm
(8572, 0, 0.0)
idx: 8572 -- playing baseline arm
(8573, 0, 0.0)
idx: 8573 -- playing baseline arm
(8574, 3, 0.0)
(8575, 0, 0.0)
idx: 8575 -- playing baseline arm
(8576, 1, 0.0)
(8577, 3, 0.0)
(8578, 0, 0.0)
idx: 8578 -- playing baseline arm
(8579, 3, 0.0)
(8580, 1, 0.0)
(8581, 3, 0.0)
(8582, 3, 0.0)
(8583, 0, 0.0)
idx: 8583 -- playing baseline arm
(8584, 0, 0.0)
idx: 8584 -- playing baseline arm
(8585, 3, 0.0)
(8586, 1, 0.0)
(8587, 0, 0.0)
idx: 8587 -- playing baseline arm
(8588, 3, 0.0)
(8589, 0, 0.0)
idx: 8589 -- playing baseline arm
(8590, 0, 0.0)
idx: 8590

(8887, 0, 0.0)
idx: 8887 -- playing baseline arm
(8888, 1, 0.0)
(8889, 1, 0.0)
(8890, 3, 0.0)
(8891, 0, 0.0)
idx: 8891 -- playing baseline arm
(8892, 1, 0.0)
(8893, 3, 0.0)
(8894, 0, 0.0)
idx: 8894 -- playing baseline arm
(8895, 3, 0.0)
(8896, 3, 0.0)
(8897, 0, 0.0)
idx: 8897 -- playing baseline arm
(8898, 1, 0.0)
(8899, 3, 0.0)
(8900, 0, 0.0)
idx: 8900 -- playing baseline arm
(8901, 3, 0.0)
(8902, 1, 0.0)
(8903, 3, 0.0)
(8904, 3, 0.0)
(8905, 1, 0.0)
(8906, 0, 0.0)
idx: 8906 -- playing baseline arm
(8907, 0, 0.0)
idx: 8907 -- playing baseline arm
(8908, 3, 0.0)
(8909, 0, 0.0)
idx: 8909 -- playing baseline arm
(8910, 1, 0.20606725665428716)
(8911, 1, 0.0259677841952205)
(8912, 1, 0.0)
(8913, 3, 0.044182169599044141)
(8914, 0, 0.0)
idx: 8914 -- playing baseline arm
(8915, 0, 0.0)
idx: 8915 -- playing baseline arm
(8916, 0, 0.0)
idx: 8916 -- playing baseline arm
(8917, 1, 0.001024705486788291)
(8918, 3, 0.0)
(8919, 0, 0.0)
idx: 8919 -- playing baseline arm
(8920, 3, 0.0)
(8921, 3, 0.0)
(8

(9177, 3, 0.0)
(9178, 3, 0.0)
(9179, 2, 0.72671914579298047)
(9180, 3, 0.0)
(9181, 0, 0.0)
idx: 9181 -- playing baseline arm
(9182, 3, 0.0)
(9183, 3, 0.0)
(9184, 3, 0.0)
(9185, 0, 0.0)
idx: 9185 -- playing baseline arm
(9186, 3, 0.0)
(9187, 0, 0.0)
idx: 9187 -- playing baseline arm
(9188, 0, 0.0)
idx: 9188 -- playing baseline arm
(9189, 1, 0.0)
(9190, 1, 0.0)
(9191, 1, 0.0)
(9192, 3, 0.0)
(9193, 3, 0.0)
(9194, 3, 0.0)
(9195, 3, 0.0)
(9196, 1, 0.0)
(9197, 3, 0.0)
(9198, 3, 0.0)
(9199, 3, 0.0)
(9200, 0, 0.0)
idx: 9200 -- playing baseline arm
(9201, 0, 0.0)
idx: 9201 -- playing baseline arm
(9202, 0, 0.0)
idx: 9202 -- playing baseline arm
(9203, 0, 0.0)
idx: 9203 -- playing baseline arm
(9204, 3, 0.0)
(9205, 1, 0.0)
(9206, 0, 0.0)
idx: 9206 -- playing baseline arm
(9207, 1, 0.0)
(9208, 0, 0.0)
idx: 9208 -- playing baseline arm
(9209, 3, 0.0)
(9210, 0, 0.0)
idx: 9210 -- playing baseline arm
(9211, 0, 0.0)
idx: 9211 -- playing baseline arm
(9212, 0, 0.0)
idx: 9212 -- playing baseline arm
(9

(9711, 3, 0.0)
(9712, 1, 0.0)
(9713, 3, 0.0)
(9714, 3, 0.0)
(9715, 0, 0.0)
idx: 9715 -- playing baseline arm
(9716, 0, 0.0)
idx: 9716 -- playing baseline arm
(9717, 0, 0.0)
idx: 9717 -- playing baseline arm
(9718, 3, 0.0)
(9719, 1, 0.0)
(9720, 0, 0.0)
idx: 9720 -- playing baseline arm
(9721, 3, 0.0)
(9722, 3, 0.0)
(9723, 1, 0.0)
(9724, 3, 0.0)
(9725, 3, 0.0)
(9726, 0, 0.0)
idx: 9726 -- playing baseline arm
(9727, 0, 0.0)
idx: 9727 -- playing baseline arm
(9728, 3, 0.0)
(9729, 1, 0.0)
(9730, 3, 0.0)
(9731, 0, 0.0)
idx: 9731 -- playing baseline arm
(9732, 0, 0.0)
idx: 9732 -- playing baseline arm
(9733, 3, 0.0)
(9734, 1, 0.0)
(9735, 0, 0.0)
idx: 9735 -- playing baseline arm
(9736, 0, 0.0)
idx: 9736 -- playing baseline arm
(9737, 1, 0.0)
(9738, 0, 0.0)
idx: 9738 -- playing baseline arm
(9739, 3, 0.0)
(9740, 3, 0.0)
(9741, 3, 0.0)
(9742, 3, 0.0)
(9743, 3, 0.0)
(9744, 3, 0.0)
(9745, 0, 0.0)
idx: 9745 -- playing baseline arm
(9746, 0, 0.0)
idx: 9746 -- playing baseline arm
(9747, 0, 0.0)
idx

## Threshold cons for comparison

In [8]:
BanditAlg = ThresholdConsBandit    
regret_thrcons = np.zeros((M, N))
arm_pulls_thrcons = np.zeros((M, N, k+1))
expt_rewards_thrcons = np.zeros((M, N))
expt_rewards_baseline_thrcons = np.zeros((M, N))

In [9]:
for m in range(M):
    params = LinearGeneratorParams(np.atleast_2d(alphas[m,:,:]), betas[m,:], d = d, k = k, intercept = intercept)
    generator = LinearGenerator(params)
    means = expected_regret_per_arm(generator)
    #Choose the baseline arm as the worst arm....
    sorted_means = np.sort(means)
    #m_idx = np.argmax(means)
    m_idx = np.where(means == sorted_means[baseline_idx])[0][0]
    base_alpha = alphas[m,m_idx,:]
    base_beta = betas[m,m_idx]
    bandit = BanditAlg(generator)
    print("Run: %d/%d"%(m+1,M))
    for i in range(N):
        (ctx, arm_idx, obs, r) = bandit.step()
        regret_thrcons[m,i] = r
        if arm_idx >= 0:
            arm_pulls_linucb[m,i,arm_idx] = 1
        else:
            arm_pulls_linucb[m,i,k] = 1
        expt_rewards_baseline_thrcons[m,i] = np.dot(ctx[1:],base_alpha) + base_beta
        expt_rewards_thrcons[m,i] = np.dot(ctx[1:], alphas[m,arm_idx,:]) + betas[m,arm_idx]


Run: 1/3
Run: 2/3
Run: 3/3


## Threshold greedy

In [ ]:
BanditAlg = ThresholdBandit    
regret_thrgre = np.zeros((M, N))
arm_pulls_thrgre = np.zeros((M, N, k+1))
expt_rewards_thrgre = np.zeros((M, N))
expt_rewards_baseline_thrgre = np.zeros((M, N))

In [ ]:
for m in range(M):
    params = LinearGeneratorParams(np.atleast_2d(alphas[m,:,:]), betas[m,:], d = d, k = k, intercept = intercept)
    generator = LinearGenerator(params)
    means = expected_regret_per_arm(generator)
    #Choose the baseline arm as the worst arm....
    sorted_means = np.sort(means)
    #m_idx = np.argmax(means)
    m_idx = np.where(means == sorted_means[baseline_idx])[0][0]
    base_alpha = alphas[m,m_idx,:]
    base_beta = betas[m,m_idx]
    bandit = BanditAlg(generator)
    print("Run: %d/%d"%(m+1,M))
    for i in range(N):
        (ctx, arm_idx, obs, r) = bandit.step()
        regret_thrgre[m,i] = r
        if arm_idx >= 0:
            arm_pulls_linucb[m,i,arm_idx] = 1
        else:
            arm_pulls_linucb[m,i,k] = 1
        expt_rewards_baseline_thrgre[m,i] = np.dot(ctx[1:],base_alpha) + base_beta
        expt_rewards_thrgre[m,i] = np.dot(ctx[1:], alphas[m,arm_idx,:]) + betas[m,arm_idx]


## Comparison of the methods

Trade-off between convergence rate, risk, and confidence level

In [ ]:
#Vary the confidence level, plot convergence rate, regret, risk
sns.tsplot(np.cumsum(regret, axis = 1), color = 'red')
sns.tsplot(np.cumsum(regret_linucb, axis = 1), color = 'blue')
sns.tsplot(np.cumsum(regret_thrgre, axis = 1), color = 'green')
sns.tsplot(np.cumsum(regret_thrcons, axis = 1), color = 'black')
plt.xlabel('round')
plt.ylabel('cumulative regret')
plt.legend(['CLUCB', 'LinUCB', 'Threshold greedy', 'Threshold cons'])
sns.despine(trim = True)

In [ ]:
#Vary the confidence level, plot convergence rate, regret, risk
sns.tsplot(np.divide(np.cumsum(regret, axis = 1), np.arange(N)), color = 'red')
sns.tsplot(np.divide(np.cumsum(regret_linucb, axis = 1),np.arange(N)), color = 'blue')
plt.xlabel('round')
plt.ylabel('per step regret')
plt.legend(['CLUCB', 'LinUCB'])
sns.despine(trim = True)

## Plot the violation of constraints........

In [ ]:
#cum_expt_rewards_rs = np.cumsum(expt_rewards_rs, axis = 1)
#cum_expt_rewards_baseline_rs = np.cumsum(expt_rewards_baseline_rs, axis = 1)
cum_expt_rewards_ucb = np.cumsum(expt_rewards_linucb, axis = 1)
cum_expt_rewards_baseline_ucb = np.cumsum(expt_rewards_baseline_linucb, axis = 1)
cum_expt_rewards_thr = np.cumsum(expt_rewards_thrgre, axis = 1)
cum_expt_rewards_baseline_thr = np.cumsum(expt_rewards_baseline_thrgre, axis = 1)
cum_expt_rewards_thrcons = np.cumsum(expt_rewards_thrcons, axis = 1)
cum_expt_rewards_baseline_thrcons = np.cumsum(expt_rewards_baseline_thrcons, axis = 1)
#cum_expt_rewards_gre = np.cumsum(expt_rewards_gre, axis = 1)
#cum_expt_rewards_baseline_gre = np.cumsum(expt_rewards_baseline_gre, axis = 1)
cum_expt_rewards_clucb = np.cumsum(expt_rewards_clucb, axis = 1)
cum_expt_rewards_baseline_clucb = np.cumsum(expt_rewards_baseline_clucb, axis = 1)

constraint_violation_clucb = np.mean((cum_expt_rewards_clucb.T - (1-alpha)*cum_expt_rewards_baseline_clucb.T)<0,0)
#constraint_violation_gre = np.mean((cum_expt_rewards_gre.T - (1-alpha_clucb)*cum_expt_rewards_baseline_gre.T)<0,0)
constraint_violation_ucb = np.mean((cum_expt_rewards_ucb.T - (1-alpha)*cum_expt_rewards_baseline_ucb.T)<0,0)
#constraint_violation_rs = np.mean((cum_expt_rewards_rs.T - (1-alpha_clucb)*cum_expt_rewards_baseline_rs.T)<0,0)
constraint_violation_thr = np.mean((cum_expt_rewards_thr.T - (1-alpha)*cum_expt_rewards_baseline_thr.T)<0,0)
constraint_violation_thrcons = np.mean((cum_expt_rewards_thrcons.T - (1-alpha)*cum_expt_rewards_baseline_thrcons.T)<0,0)

all_method_constraints = 100*np.stack((constraint_violation_ucb,
                                       constraint_violation_thr,
                                       constraint_violation_thrcons,
                                       constraint_violation_clucb))

#all_method_constraints = 100*np.stack((constraint_violation_ucb,
#                                       constraint_violation_gre,
#                                       constraint_violation_thr,
#                                       constraint_violation_thrcons,
#                                       constraint_violation_rs,
#                                       constraint_violation_clucb))

#df_constraints = pd.DataFrame(data = all_method_constraints.T, columns = ['LinUCB', 'Greedy', 'RS-greedy', 'RS-conservative', 'RS-LinUCB', 'CLUCB'], index = range(M))
df_constraints = pd.DataFrame(data = all_method_constraints.T, columns = ['LinUCB', 'RS-greedy', 'RS-conservative', 'CLUCB'], index = range(M))

In [ ]:
sns.barplot(data = df_constraints)
plt.xlabel("Method")
plt.ylabel("Percentage time violating baseline performance constraint")
sns.despine(trim = True)

In [ ]:
#cum_expt_rewards_clucb = np.cumsum(expt_rewards_clucb, axis = 1)
#cum_expt_rewards_baseline_clucb = np.cumsum(expt_rewards_baseline_clucb, axis = 1)

#plt.plot(cum_expt_rewards_clucb.T)
#cum_expt_rewards_clucb.shape
plt.plot(cum_expt_rewards_clucb.T - (1-alpha)*cum_expt_rewards_baseline_clucb.T);